In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import time

url = requests.get("http://salaryguide.diamondbacklab.com/#/salGuide")
soup = bs(url.content, 'lxml')

In [2]:
from selenium import webdriver

path_to_chromedriver = 'D:\Programming\Scraping\Drivers\chromedriver'
browser = webdriver.Chrome(executable_path = path_to_chromedriver)

In [3]:
url = 'http://salaryguide.diamondbacklab.com/#/'
browser.get(url)

In [4]:
browser.find_element_by_id('view').click() # click view
time.sleep(5)

In [5]:
browser.find_element_by_id('search').send_keys("CMNS-Computer Science") # only get compsci

In [6]:
row = browser.find_element_by_tag_name('table')

In [7]:
entries = row.text.splitlines()
entries

['EMPLOYEE ▲ DEPARTMENT TITLE SALARY',
 'Adams, Brandi Kristine CMNS-Computer Science Assoc Dir $83,287.26',
 'Agrawala, Ashok K. CMNS-Computer Science Prof $111,685.34',
 'Aloimonos, John CMNS-Computer Science Prof $80,564.98',
 'Atchison, Kate L CMNS-Computer Science Coordinator $53,000.00',
 'Bhattacharjee, Samrat CMNS-Computer Science Prof $191,698.92',
 'Boykin, Regis D. CMNS-Computer Science Accounting Assoc $36,006.20',
 'Carpuat, Marine CMNS-Computer Science Asst Prof $56,107.00',
 'Chen, Ray Sun CMNS-Computer Science Fac Spec $92,058.24',
 'Chick, Brenda K. CMNS-Computer Science Coordinator $56,637.07',
 'Childs, Andrew M CMNS-Computer Science Assoc Prof $67,034.00']

In [8]:
columns = entries[0]
columns

'EMPLOYEE ▲ DEPARTMENT TITLE SALARY'

In [9]:
columns = columns[:9] + columns[11:]
columns

'EMPLOYEE DEPARTMENT TITLE SALARY'

In [10]:
columns = columns.split(' ')
columns

['EMPLOYEE', 'DEPARTMENT', 'TITLE', 'SALARY']

In [11]:
df = pd.DataFrame(columns = columns)

In [12]:
import re

In [13]:
regex = re.compile('^(.+) CMNS-Computer Science (.+) \$(.+)$')

In [14]:
def get_entries(entries, df):
    for entry in entries[1:]:
        m = regex.match(entry)
        df1 = pd.DataFrame([[m.group(1), 'CMNS-Computer Science', m.group(2), m.group(3)]], columns = columns)
        df = df.append(df1)
    return df

In [15]:
df = get_entries(entries, df)

In [16]:
def scrape_each_page(df):
    page = ''
    prev_page = ''
    while(True):
        browser.find_element_by_id('next').click()
        row = browser.find_element_by_tag_name('table')
        prev_page = page
        page = row.text
        if(page == prev_page):
            break
        entries = row.text.splitlines()
        df = get_entries(entries, df)
    return df
df = scrape_each_page(df)
df.head()

,EMPLOYEE,DEPARTMENT,TITLE,SALARY
0,"Adams, Brandi Kristine",CMNS-Computer Science,Assoc Dir,"83,287.26"
0,"Agrawala, Ashok K.",CMNS-Computer Science,Prof,"111,685.34"
0,"Aloimonos, John",CMNS-Computer Science,Prof,"80,564.98"
0,"Atchison, Kate L",CMNS-Computer Science,Coordinator,"53,000.00"
0,"Bhattacharjee, Samrat",CMNS-Computer Science,Prof,"191,698.92"


In [17]:
df['year'] = 2017
df.head()

,EMPLOYEE,DEPARTMENT,TITLE,SALARY,year
0,"Adams, Brandi Kristine",CMNS-Computer Science,Assoc Dir,"83,287.26",2017
0,"Agrawala, Ashok K.",CMNS-Computer Science,Prof,"111,685.34",2017
0,"Aloimonos, John",CMNS-Computer Science,Prof,"80,564.98",2017
0,"Atchison, Kate L",CMNS-Computer Science,Coordinator,"53,000.00",2017
0,"Bhattacharjee, Samrat",CMNS-Computer Science,Prof,"191,698.92",2017


In [ ]:
browser.find_element_by_id('previous').click() # toggle to see other charts

In [ ]:
for n in range(2013,2017):
    time.sleep(6)
    link = '#/salGuide?year=' + str(n)
    browser.find_element_by_xpath("//a[@href=\'{0}\']".format(link)).click()
    time.sleep(6)
    browser.find_element_by_id('search').send_keys("CMNS-Computer Science")
    df1 = pd.DataFrame(columns = columns)
    df1 = scrape_each_page(df1)
    df1['year'] = n
    df = df.append(df1)
df    